In [ ]:
#Importando as bibliotecas importantes do keras
from tensorflow.keras import layers
from tensorflow.keras import Model
import os

from google.colab import drive
drive.mount('/content/gdrive')
base_dir = '/content/gdrive/My Drive/Colab Notebooks/dataset/'
train_dir = os.path.join(base_dir, 'train_set')
validation_dir = os.path.join(base_dir, 'test_set')

#Para ver se deu certo imprimir os nomes das pastas:

print(train_dir)
print(validation_dir)

# Imagens benigno no treinamento:
train_benigno_dir = os.path.join(train_dir, 'benigno')

# Imagens maligno no treinamento:
train_maligno_dir = os.path.join(train_dir, 'maligno')

# Imagens benigno teste:
validation_benigno_dir = os.path.join(validation_dir, 'benigno')

# Imagens maligno no teste:
validation_maligno_dir = os.path.join(validation_dir, 'maligno')

#Testar se o código ta dando certo (imprimir os 10 primeiros nomes na pasta de treinamento):

train_benigno_fnames = os.listdir(train_benigno_dir)
train_benigno_fnames.sort()
print(train_benigno_fnames[:10])

train_maligno_fnames = os.listdir(train_maligno_dir)
train_maligno_fnames.sort()
print(train_maligno_fnames[:10])

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# Preparando os dados ( lotes de imagens e seus labels) a partir dos diretórios com as imagens

batch_size = 32

# Augmentation para o treinamento
train_datagen = ImageDataGenerator(
          rescale =1./255,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True)


# Augmentation para o teste (apenas rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


# Leitura das Imagens
train_generator = train_datagen.flow_from_directory(
      '/content/gdrive/My Drive/Colab Notebooks/dataset/train_set',
      target_size=(50,50), # tamanho das imagens
      batch_size = batch_size,
      shuffle = True,
      class_mode='binary') # labels binários


test_generator = test_datagen.flow_from_directory(
      '/content/gdrive/My Drive/Colab Notebooks/dataset/test_set',
      target_size=(50,50), # tamanho das imagens
      batch_size = 1,
      class_mode='binary') # labels binários

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import model_from_json
from keras import backend as K
import numpy as np


K.set_image_dim_ordering('th')

model = Sequential()
# Primeira parte com convoluções e maxpooling
model.add(Conv2D(32,(3,3), input_shape=(3,50,50)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda parte com convoluções e maxpooling
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Terceira parte com convoluções e maxpooling
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Flatten
model.add(Flatten())

# Rede conectada com duas camadas 
model.add(Dense(64)) # 64 neurônios
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) # 1 neurônio para clasificação
model.add(Activation('sigmoid')) # Função de ativação sigmoid

model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])


validation_benigno_fnames = os.listdir(validation_benigno_dir)
validation_maligno_fnames = os.listdir(validation_maligno_dir)

model.fit_generator(
    train_generator,
    steps_per_epoch=8000//batch_size,
    epochs=50,
       
)

model_json = model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")





Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/dataset/train_set
/content/gdrive/My Drive/Colab Notebooks/dataset/test_set
['8863_idx5_x1001_y1001_class0.png', '8863_idx5_x1001_y1051_class0.png', '8863_idx5_x1001_y1101_class0.png', '8863_idx5_x1001_y1151_class0.png', '8863_idx5_x1001_y1201_class0.png', '8863_idx5_x1001_y1251_class0.png', '8863_idx5_x1001_y1301_class0.png', '8863_idx5_x1001_y1351_class0.png', '8863_idx5_x1001_y1401_class0.png', '8863_idx5_x1001_y1451_class0.png']
['8863_idx5_x1001_y1501_class1.png', '8863_idx5_x1001_y1551_class1.png', '8863_idx5_x1001_y1601_class1.png', '8863_idx5_x1001_y801_class1.png', '8863_idx5_x1001_y851_class1.png', '8863_idx5_x1001_y901_class1.png', '8863_idx5_x1001_y951_class1.png', '8863_idx5_x1051_y1001_class1.png', '8863_idx5_x1051_y1051_class1.png', '8863_idx5_x1051_y1101_class1.png']
Found 21881 images belonging to 2 

In [ ]:
model.evaluate_generator(test_generator, steps=2000)

In [ ]:
# Carregando o arquivo .json e criando o modelo
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#Carregando os pesos para o modelo
loaded_model.load_weights('model.h5')
print('Loaded model from disk')

#Avaliando o modelo carregado no conjunto de teste
loaded_model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

score = loaded_model.evaluate_generator(test_generator,steps=2000)

print("{}: {}".format(loaded_model.metrics_names[1],score[1]*100))
print("{}: {}".format(loaded_model.metrics_names[0],score[0]*100))

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 32, 48, 48)        896       
_________________________________________________________________
activation_51 (Activation)   (None, 32, 48, 48)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 32, 24, 24)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 32, 22, 22)        9248      
_________________________________________________________________
activation_52 (Activation)   (None, 32, 22, 22)        0         
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 32, 11, 11)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 64, 9, 9)          18496     
__________